In [4]:
text_en = """
The packgae has been delivered but is damaged (photo:
https://i.postimg.cc/TPr4LQwY/MEA2-1.jpg). When i openned iy , 
every flask was broke :( I put 1 star because I cannot put 
-100 starts #wtf >>>>>>>>>>> dont buy this brand $$$$ \U0001F643
"""

text_fr = """
Le colsi a été livré mais cabossé (photo :
https://i.postimg.cc/TPr4LQwY/MEA2-1.jpg). Quan j'ai ouvert , 
tous les flacons été casser :( abusé je met 1 étoiles parce 
que je peux pas mettre -100 étoiles #labus >>>>>>>>>>> 
plus jamais cette marque $$$$ \U0001F643
"""

Dans ce contecte, nous avons 2 review de notre produit amazon, 1 en francais et un en anglais.

Nous allons essayer de le normaliser et de le netoyer.

In [ ]:
!python -m pip install emot torch transformers sentencepiece autocorrect clean-text textblob wordfreq

# Corrections par approches déterministes

Il existe différentes façon d'effectuer des corrections 
- Les approches algorithmiques déterministes
- Les approches de machine learning
    - Aller chercher les models pré-existant présent sur HuggingFace 😁
- Les approches linguistiques
- La combinaisons des 3 

In [7]:
# Approche déterministes
from autocorrect import Speller

spell_en = Speller('en')
spell_fr = Speller('fr')

print(spell_en('The packgae has been delivered but is damaged (photo:'))
print(spell_fr('Le colsi a été livré mais cabossé (photo :'))

The package has been delivered but is damaged (photo:
Le cols a été livré mais cabossé (photo :


In [13]:
!python -m pip install protobuf
# Approches par le machine learning 
# Plus précisement avec les transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('Bhuvana/t5-base-spellchecker')
model = AutoModelForSeq2SeqLM.from_pretrained('Bhuvana/t5-base-spellchecker')

In [18]:
def correcter(inputs: str, tokenizer: AutoTokenizer, model: AutoModelForSeq2SeqLM):
    input_ids = tokenizer.encode(inputs, return_tensors='pt')
    sample_output = model.generate(
        input_ids,
        do_sample               = True,
        max_length              = 50,
        top_p                   = 0.99,
        num_return_sequences    = 1
    )
    res = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return res

In [21]:
print(correcter('christmas is celbrated on decembr 25 evry ear', tokenizer, model))
print(correcter('When i openned iy, every flask was broke', tokenizer, model))

christmas is celebrated on december 25 every year
i opened it all flask was broken


Comme on peut le voir, cela ne marche pas encore super bien, les recherches sont toujours en cours et les models les plus avancée sont garder secret 

#### Info du cours, en 2022/2021 aujourd'hui il y a des models plus puissants

In [1]:
from transformers import pipeline

fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

/opt/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.74k/1.74k [00:00<00:00, 12.4MB/s]
model.safetensors: 100%|██████████| 558M/558M [00:15<00:00, 36.5MB/s] 
tokenizer_config.json: 100%|██████████| 353/353 [00:00<00:00, 2.30MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 2.39MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.84MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 12.1MB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 1.91MB/s]


In [2]:
print(fix_spelling('christmas is celbrated on decembr 25 evry ear'))
print(fix_spelling('When i openned iy, every flask was broke')) # Devrait etre was broken / were broke

/opt/NLP/lib/python3.11/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Christmas is celebrated on December 25 every year.'}]
[{'generated_text': 'When I opened it, every flask was broke.'}]
